# Granite Plan-Solve Agent

This notebook demonstrates a comprehensive **Plan-Solve Agent** that can break down complex tasks into executable steps, execute them using the function calling agent, and adapt the plan based on results.

A [Plan-Solve Agent](https://arxiv.org/abs/2305.04091) follows these steps:

1. **Planning Phase**: The agent analyzes a user request and creates a step-by-step plan
2. **Execution Phase**: The agent executes each step using function calling with available tools
3. **Replanning Phase**: The agent reviews progress and updates the plan based on results
4. **Iteration**: The cycle continues until the task is complete

This approach allows the agent to handle complex, multi-step tasks that require:

- Breaking down complex requests into manageable steps
- Using external APIs and tools to gather information
- Adapting the plan based on intermediate results
- Coordinating multiple tool calls to achieve the final goal


## Plan-Solve Agent Implementation Steps

This notebook will guide you through building a comprehensive Plan-Solve Agent with the following steps:

1. **Environment Setup**: Configure watsonx.ai credentials and install dependencies
   
    **Model Initialization**: Set up Granite models for planning and execution

   
    **Import Modules**: Import tools, states, prompts, and nodes from the modular structure
   
3. **Graph Construction**: Build the LangGraph workflow following the plan_solve_agent.py structure

    **Graph Visualization**: Display the agent's workflow as a visual diagram

   
5. **Agent Execution**: Run examples and demonstrate the agent's capabilities
6. **Display Results**: Show the matplotlib figure generated by the agent


## Step 1. Environment Setup

This Plan-Solve Agent requires several components to function properly:

### Prerequisites
- **Python 3.11+**: The notebook is compatible with Python 3.11 and 3.12
- **Jupyter Notebook**: Best suited for interactive development and visualization
- **watsonx.ai Access**: You'll need credentials for IBM's watsonx.ai platform
- **API Keys**: Optional API keys for weather and stock market data (with fallback demo data)

### Required Credentials
You'll need to set up the following environment variables:
- `WATSONX_URL`: Your watsonx.ai endpoint URL
- `WATSONX_APIKEY`: Your watsonx.ai API key  
- `WATSONX_PROJECT_ID`: Your watsonx.ai project ID
- `WEATHER_API_KEY`: (Optional) OpenWeatherMap API key for real weather data
- `AV_STOCK_API_KEY`: (Optional) Alpha Vantage API key for real stock data

In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import Image, display
load_dotenv()

from chains.function_calling_llm import fc_llm
from chains.plannig_llm import planning_llm
from states import PlanSolve
from tools.weather_tools import (
    get_current_weather,
    get_geo_coordinates,
    get_weather_forecast,
    plot_weather_timeseries,
)
from tools.stock_market_tools import get_stock_price
from tools.plan_complete import plan_complete
from nodes.planner_nodes import planner_node, replanner_node, should_end
from nodes.execute_step_node import execute_step
from langgraph.graph import END, START, StateGraph
from chains.plannig_llm import planning_llm

In [ ]:
# Check Python version
!python --version

## Step 2: Graph Construction


In [ ]:
# Create the Plan-Solve Agent graph
graph = StateGraph(PlanSolve)

# Add nodes to the graph
graph.add_node("planner_node", planner_node)
graph.add_node("function_calling_agent", execute_step)
graph.add_node("replan", replanner_node)

# Add edges to connect nodes
graph.add_edge(START, "planner_node")
graph.add_edge("planner_node", "function_calling_agent")
graph.add_edge("function_calling_agent", "replan")

# Add conditional edges for routing
graph.add_conditional_edges(
    "replan",
    should_end,
    {"function_calling_agent": "function_calling_agent", END: END},
)

# Compile the graph
planner_agent = graph.compile()

In [ ]:
# Show the agent
display(Image(planner_agent.get_graph(xray=True).draw_mermaid_png()))

## Available Tools

The agent has access to various tools including:
- **Weather Tools**: Get current weather, coordinates, forecasts, and create visualizations
- **Stock Market Tools**: Retrieve stock price information
- **Planning Tools**: Complete plan execution

The agent uses LangGraph to orchestrate this complex workflow, demonstrating advanced agentic AI capabilities.

### Let's demonstrate how to call the internal tools directly to understand their functionality:

In [ ]:
from tools.weather_tools import (
    _get_geo_coordinates_internal,
    _get_weather_forecast_internal,
    _plot_weather_timeseries_internal
)

# Example 1: Get coordinates for Yorktown
print("📍 Getting coordinates for Yorktown, New York, US...")
yorktown_lat, yorktown_lon = _get_geo_coordinates_internal("Yorktown", "New York", "US")
print(f"Yorktown coordinates: ({yorktown_lat}, {yorktown_lon})")

# Example 2: Get coordinates for Armonk
print("\n📍 Getting coordinates for Armonk, New York, US...")
armonk_lat, armonk_lon = _get_geo_coordinates_internal("Armonk", "New York", "US")
print(f"Armonk coordinates: ({armonk_lat}, {armonk_lon})")

In [ ]:
# Example 3: Get weather forecast for Yorktown
print("🌤️ Getting weather forecast for Toronto...")
yorktown_forecast = _get_weather_forecast_internal(yorktown_lat, yorktown_lon)
print(f"Yorktown forecast: {len(yorktown_forecast)} data points")
print(f"Sample data: {yorktown_forecast[:3]}")

# Example 4: Get weather forecast for Armonk
print("\n🌤️ Getting weather forecast for Armonk...")
armonk_forecast = _get_weather_forecast_internal(armonk_lat, armonk_lon)
print(f"Armonk forecast: {len(armonk_forecast)} data points")
print(f"Sample data: {armonk_forecast[:3]}")

In [ ]:
# Example 5: Create a comparison plot using internal function
print("📊 Creating weather comparison plot...")

# Prepare data for plotting
weather_data = {
    "Yorktown": yorktown_forecast,
    "Armonk": armonk_forecast
}

# Create the plot using internal function
_plot_weather_timeseries_internal(
    weather_data=weather_data,
    title="Weather Forecast Comparison - Yorktown vs Armonk",
    save_path="yorktown_armonk_comparison.png"
)


## Step 3: Execute the Plan-Solve Agent

Now let's run the Plan-Solve Agent with a complex weather forecast comparison task. This will demonstrate how the agent:

1. **Plans**: Breaks down the request into steps
2. **Executes**: Uses tools to gather data
3. **Replans**: Adapts based on results
4. **Completes**: Generates the final visualization

Let's execute the agent with a weather forecast comparison:


In [ ]:
# Execute the Plan-Solve Agent with a weather forecast comparison task
user_input = "compare weather forecast for Yorktown and Armonk"

print("Starting Plan-Solve Agent execution...")
print(f"User request: {user_input}")


# Execute the agent
result = planner_agent.invoke({"input": user_input})

print("Plan-Solve Agent execution completed!")
print(f"Final result: {result.get('response', 'No response generated')}")

## Step 4: Display the Generated Matplotlib Figure

The Plan-Solve Agent has executed and should have generated a matplotlib figure showing the weather forecast comparison. Let's check if the figure was created and display it:


In [ ]:
from tools import ROOT_DIR
display(Image(filename=f'{ROOT_DIR}/images/plot.png'))

## Summary: Plan-Solve Agent Capabilities

The Plan-Solve Agent demonstrates several capabilities:

### **Planning & Replanning**
- **Initial Planning**: Breaks down complex requests into executable steps
- **Adaptive Replanning**: Updates plans based on execution results
- **Dynamic Decision Making**: Decides when to continue or complete tasks

### **Tool Integration**
- **Weather Tools**: Get coordinates, forecasts, and create visualizations
- **Stock Market Tools**: Retrieve financial data
- **Planning Tools**: Complete plan execution
- **Function Calling**: Function Calling Agent built in this repo
